In [1]:
import sys
import time
import random

import numpy as np
import gym
from gym_chess import ChessEnvV1
import gym_chess
import copy
import pandas as pd
import numpy as np
import torch
from torch import nn
from processing import *


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/Users/call

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/Users/call

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/Users/call

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/calla/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/Users/call

AttributeError: _ARRAY_API not found

/Users/calla/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
#le NN prend comme entrée l'état et en sortie un array de 64 * 64 + 4 elements qui correspond aux 64 * 64 + 4 coups possibles

# Taille des entrées et sorties
input_size = 8 * 8  # État de l'échiquier aplati
output_size = 64 * 64 + 4  # 4100 coups possibles

# Modèle du réseau
model = nn.Sequential(
    nn.Flatten(),                     # Aplatit l'entrée (8x8 -> 64)
    nn.Linear(input_size, 256),       # Première couche cachée avec 256 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(256, 512),              # Deuxième couche cachée avec 512 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(512, 1024),             # Troisième couche cachée avec 1024 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(1024, 512),             # Quatrième couche cachée avec 512 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(512, 256),              # Cinquième couche cachée avec 256 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(256, output_size),      # Couche de sortie avec 4100 neurones
    nn.Softmax(dim=-1)                # Fonction Softmax pour des probabilités sur les coups possibles
)

# Afficher le modèle
print(model)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=64, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=1024, bias=True)
  (6): ReLU()
  (7): Linear(in_features=1024, out_features=512, bias=True)
  (8): ReLU()
  (9): Linear(in_features=512, out_features=256, bias=True)
  (10): ReLU()
  (11): Linear(in_features=256, out_features=4100, bias=True)
  (12): Softmax(dim=-1)
)


In [3]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lr=0.001, params = model.parameters())

### Data loading and Data cleaning

In [4]:
df = pd.read_csv("games.csv")

In [5]:
display(df.head())

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [6]:
display(df.columns)

Index(['id', 'rated', 'created_at', 'last_move_at', 'turns', 'victory_status',
       'winner', 'increment_code', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply'],
      dtype='object')

In [7]:
df = df[df["white_rating"] > 2000]
df = df[df["black_rating"] > 2000]

In [8]:
display(df.count())

id                849
rated             849
created_at        849
last_move_at      849
turns             849
victory_status    849
winner            849
increment_code    849
white_id          849
white_rating      849
black_id          849
black_rating      849
moves             849
opening_eco       849
opening_name      849
opening_ply       849
dtype: int64

In [9]:
display(df.iloc[0]["moves"])

'e4 e5 Nf3 Nc6 Nc3 Nf6 Nxe5 Nxe5 d4 Nc6 d5 Bb4 dxc6 Nxe4 Qd4 Qe7 Be3 O-O cxd7 Bxd7 Bd3 Bc6 a3 Bxc3+ bxc3 Rad8 Qb4 Qe5 O-O-O a5 Qc4 Qxc3 Qxc3 Nxc3 Rde1 Bxg2 Rhg1 Bf3 Bh6 g6 Bxf8 Kxf8 Re3 Bc6 h4 Nd5 Re5 f6 Re4 Kf7 Rge1 Kg7 c4 Nb6 Re7+ Kf8 Kc2 Rd7 Re8+ Kf7 h5 Ba4+ Kb2 gxh5 R8e3 Bc6 Kc2 Kg7 Re7+ Kh6 Rxd7 Bxd7 Re7 h4 c5 Ba4+ Kd2 Nd5 Re4 Bc6 Rxh4+ Kg5 Rxh7 Kf4 Rxc7 Nxc7'

In [10]:

env = ChessEnvV1(player_color="WHITE", opponent="none", log=True)

#
# Play against self
#
num_episodes = 2
num_steps = 100

total_steps = 0
collected_rewards = []
start = time.time()

In [ ]:


for i in range(num_episodes):
    initial_state = env.reset()
    print("\n", "=" * 10, "NEW GAME", "=" * 10)
    env.render()
    total_rewards = {"WHITE": 0, "BLACK": 0}
    for _, game in df.iterrows():
        #On récupère les coups
        chess_moves = game["moves"]
        
        #on transforme le str en une liste de coups
        moves_list = chess_moves.split()
        v = 0
        #on iter sur les coups sous pgn notation
        for move_pgn in moves_list:
            if '+' in move_pgn:
                move_pgn = move_pgn[:-1]

            print(f"{v} ème itération")
            #on récupère l'état
            state = env.state
            print(f"state {state}")
                
            print(f"player to move {env.player}")
            
            state = torch.tensor(state, dtype=torch.float32)
            
            if len(state.shape) == 2:  # Assuming shape is (8, 8)
                state = state.unsqueeze(0)
            
            #On réalise la prédiction
            output_vector = model(state)
            
            #on transforme output_vector en action
            output_action = torch.argmax(output_vector).item()
            
            #On récupère la liste des coups possibles
            possible_moves_copy = copy.deepcopy(env.possible_moves)
            
            #On transforme les coups en pgn
            possible_moves_pgn = list_move_to_string(possible_moves_copy, env)
            #print(possible_moves_pgn)
            print(f"possible moves pgn {possible_moves_pgn} lenght {len(possible_moves_pgn)}")
            cleaned_moves = clean_moves(possible_moves_pgn)
            print(f"cleaned moves {cleaned_moves} lenght {len(cleaned_moves)}")
            
            env.render()
            #print(cleaned_moves)
            
            display(env.possible_moves)
            #On récupère l'indice de move_pgn dans possible_moves_pgn

            try:
                index = cleaned_moves.index(move_pgn)
            except ValueError:
                index = find_most_similar_move_char_overlap(move_pgn, cleaned_moves)

            if index is None:
                print(f"Attention: Aucun coup similaire trouvé pour {move_pgn} dans {cleaned_moves}. Coup sauté.")
                v += 1
                break

            #On récupère le bon coup
            m = env.possible_moves[index]


            #On le transforme en action
            a = env.move_to_action(m)
            print(f"m:{m}, a:{a}, index:{index}, move_pgn:{move_pgn}")
            
            action_one_hot = torch.zeros(64 * 64 + 4)  # Crée un tenseur rempli de 0
            action_one_hot[a] = 1  
            
            #On calcul la loss
            loss = loss_function(output_vector, action_one_hot)
            
            #On met à jour le réseau
            optimizer.zero_grad() # Mettre les gradients à 0
            loss.backward()
            optimizer.step()


            # perform action
            state, reward, done, _ = env.step(a)
            v += 1
            
            if done or v > num_steps:
                break



 ========== NEW GAME ==========
    -------------------------
 8 |  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖ |
 7 |  ♙  ♙  ♙  ♙  ♙  ♙  ♙  ♙ |
 6 |  .  .  .  .  .  .  .  . |
 5 |  .  .  .  .  .  .  .  . |
 4 |  .  .  .  .  .  .  .  . |
 3 |  .  .  .  .  .  .  .  . |
 2 |  ♟  ♟  ♟  ♟  ♟  ♟  ♟  ♟ |
 1 |  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜ |
    -------------------------
      a  b  c  d  e  f  g  h 
0 ème itération
state [[-3 -5 -4 -2 -1 -4 -5 -3]
 [-6 -6 -6 -6 -6 -6 -6 -6]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 6  6  6  6  6  6  6  6]
 [ 3  5  4  2  1  4  5  3]]
player to move WHITE
possible moves pgn ['a2a3', 'a2a4', 'b2b3', 'b2b4', 'c2c3', 'c2c4', 'd2d3', 'd2d4', 'e2e3', 'e2e4', 'f2f3', 'f2f4', 'g2g3', 'g2g4', 'h2h3', 'h2h4', 'Nb1a3', 'Nb1c3', 'Ng1f3', 'Ng1h3'] lenght 20
cleaned moves ['a3', 'a4', 'b3', 'b4', 'c3', 'c4', 'd3', 'd4', 'e3', 'e4', 'f3', 'f4', 'g3', 'g4', 'h3', 'h4', 'Na3', 'Nc3', 'Nf3', 'Nh3'] lenght 20
    ----------------------

[[array([6, 0], dtype=int8), array([5, 0], dtype=int8)],
 [array([6, 0], dtype=int8), array([4, 0], dtype=int8)],
 [array([6, 1], dtype=int8), array([5, 1], dtype=int8)],
 [array([6, 1], dtype=int8), array([4, 1], dtype=int8)],
 [array([6, 2], dtype=int8), array([5, 2], dtype=int8)],
 [array([6, 2], dtype=int8), array([4, 2], dtype=int8)],
 [array([6, 3], dtype=int8), array([5, 3], dtype=int8)],
 [array([6, 3], dtype=int8), array([4, 3], dtype=int8)],
 [array([6, 4], dtype=int8), array([5, 4], dtype=int8)],
 [array([6, 4], dtype=int8), array([4, 4], dtype=int8)],
 [array([6, 5], dtype=int8), array([5, 5], dtype=int8)],
 [array([6, 5], dtype=int8), array([4, 5], dtype=int8)],
 [array([6, 6], dtype=int8), array([5, 6], dtype=int8)],
 [array([6, 6], dtype=int8), array([4, 6], dtype=int8)],
 [array([6, 7], dtype=int8), array([5, 7], dtype=int8)],
 [array([6, 7], dtype=int8), array([4, 7], dtype=int8)],
 [array([7, 1], dtype=int8), array([5, 0], dtype=int8)],
 [array([7, 1], dtype=int8), ar

m:[array([6, 4], dtype=int8), array([4, 4], dtype=int8)], a:3364, index:9, move_pgn:e4
           >>>>>>>>>> WHITE
    -------------------------
 8 |  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖ |
 7 |  ♙  ♙  ♙  ♙  ♙  ♙  ♙  ♙ |
 6 |  .  .  .  .  .  .  .  . |
 5 |  .  .  .  .  .  .  .  . |
 4 |  .  .  .  .  .  .  .  . |
 3 |  .  .  .  .  .  .  .  . |
 2 |  ♟  ♟  ♟  ♟  ♟  ♟  ♟  ♟ |
 1 |  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜ |
    -------------------------
      a  b  c  d  e  f  g  h 
1 ème itération
state [[-3 -5 -4 -2 -1 -4 -5 -3]
 [-6 -6 -6 -6 -6 -6 -6 -6]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 6  6  6  6  0  6  6  6]
 [ 3  5  4  2  1  4  5  3]]
player to move WHITE
possible moves pgn ['Nb8a6', 'Nb8c6', 'Ng8f6', 'Ng8h6', 'a7a6', 'a7a5', 'b7b6', 'b7b5', 'c7c6', 'c7c5', 'd7d6', 'd7d5', 'e7e6', 'e7e5', 'f7f6', 'f7f5', 'g7g6', 'g7g5', 'h7h6', 'h7h5'] lenght 20
cleaned moves ['Na6', 'Nc6', 'Nf6', 'Nh6', 'a6', 'a5', 'b6', 'b5', 'c6', 'c5', 'd6', 'd5', 'e

/Users/calla/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([4100])) that is different to the input size (torch.Size([1, 4100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[[array([0, 1], dtype=int8), array([2, 0], dtype=int8)],
 [array([0, 1], dtype=int8), array([2, 2], dtype=int8)],
 [array([0, 6], dtype=int8), array([2, 5], dtype=int8)],
 [array([0, 6], dtype=int8), array([2, 7], dtype=int8)],
 [array([1, 0], dtype=int8), array([2, 0], dtype=int8)],
 [array([1, 0], dtype=int8), array([3, 0], dtype=int8)],
 [array([1, 1], dtype=int8), array([2, 1], dtype=int8)],
 [array([1, 1], dtype=int8), array([3, 1], dtype=int8)],
 [array([1, 2], dtype=int8), array([2, 2], dtype=int8)],
 [array([1, 2], dtype=int8), array([3, 2], dtype=int8)],
 [array([1, 3], dtype=int8), array([2, 3], dtype=int8)],
 [array([1, 3], dtype=int8), array([3, 3], dtype=int8)],
 [array([1, 4], dtype=int8), array([2, 4], dtype=int8)],
 [array([1, 4], dtype=int8), array([3, 4], dtype=int8)],
 [array([1, 5], dtype=int8), array([2, 5], dtype=int8)],
 [array([1, 5], dtype=int8), array([3, 5], dtype=int8)],
 [array([1, 6], dtype=int8), array([2, 6], dtype=int8)],
 [array([1, 6], dtype=int8), ar

...

m:[array([3, 4], dtype=int8), array([4, 4], dtype=int8)], a:1828, index:1, move_pgn:e4
0 ème itération
state [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
player to move WHITE
possible moves pgn ['f7f8', 'Ke5e4', 'Ke5e6', 'Ke5f5', 'Ke5d5', 'Ke5d4', 'Ke5f6', 'Ke5d6'] lenght 8
cleaned moves ['f8', 'Ke4', 'Ke6', 'Kf5', 'Kd5', 'Kd4', 'Kf6', 'Kd6'] lenght 8
    -------------------------
 8 |  .  .  .  .  .  .  .  . |
 7 |  .  .  .  .  .  ♟  .  . |
 6 |  .  .  .  .  .  .  .  . |
 5 |  .  .  .  .  ♚  .  .  . |
 4 |  .  .  .  .  .  .  .  . |
 3 |  .  .  .  .  .  .  ♔  . |
 2 |  .  .  .  .  .  .  .  . |
 1 |  .  .  .  .  .  .  .  . |
    -------------------------
      a  b  c  d  e  f  g  h 


[[array([1, 5], dtype=int8), array([0, 5], dtype=int8)],
 [array([3, 4], dtype=int8), array([4, 4], dtype=int8)],
 [array([3, 4], dtype=int8), array([2, 4], dtype=int8)],
 [array([3, 4], dtype=int8), array([3, 5], dtype=int8)],
 [array([3, 4], dtype=int8), array([3, 3], dtype=int8)],
 [array([3, 4], dtype=int8), array([4, 3], dtype=int8)],
 [array([3, 4], dtype=int8), array([2, 5], dtype=int8)],
 [array([3, 4], dtype=int8), array([2, 3], dtype=int8)]]

m:[array([3, 4], dtype=int8), array([4, 4], dtype=int8)], a:1828, index:1, move_pgn:e4
0 ème itération
state [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
player to move WHITE
possible moves pgn ['f7f8', 'Ke5e4', 'Ke5e6', 'Ke5f5', 'Ke5d5', 'Ke5d4', 'Ke5f6', 'Ke5d6'] lenght 8
cleaned moves ['f8', 'Ke4', 'Ke6', 'Kf5', 'Kd5', 'Kd4', 'Kf6', 'Kd6'] lenght 8
    -------------------------
 8 |  .  .  .  .  .  .  .  . |
 7 |  .  .  .  .  .  ♟  .  . |
 6 |  .  .  .  .  .  .  .  . |
 5 |  .  .  .  .  ♚  .  .  . |
 4 |  .  .  .  .  .  .  .  . |
 3 |  .  .  .  .  .  .  ♔  . |
 2 |  .  .  .  .  .  .  .  . |
 1 |  .  .  .  .  .  .  .  . |
    -------------------------
      a  b  c  d  e  f  g  h 


[[array([1, 5], dtype=int8), array([0, 5], dtype=int8)],
 [array([3, 4], dtype=int8), array([4, 4], dtype=int8)],
 [array([3, 4], dtype=int8), array([2, 4], dtype=int8)],
 [array([3, 4], dtype=int8), array([3, 5], dtype=int8)],
 [array([3, 4], dtype=int8), array([3, 3], dtype=int8)],
 [array([3, 4], dtype=int8), array([4, 3], dtype=int8)],
 [array([3, 4], dtype=int8), array([2, 5], dtype=int8)],
 [array([3, 4], dtype=int8), array([2, 3], dtype=int8)]]

m:[array([3, 4], dtype=int8), array([4, 4], dtype=int8)], a:1828, index:1, move_pgn:e4
0 ème itération
state [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
player to move WHITE
possible moves pgn ['f7f8', 'Ke5e4', 'Ke5e6', 'Ke5f5', 'Ke5d5', 'Ke5d4', 'Ke5f6', 'Ke5d6'] lenght 8
cleaned moves ['f8', 'Ke4', 'Ke6', 'Kf5', 'Kd5', 'Kd4', 'Kf6', 'Kd6'] lenght 8
    -------------------------
 8 |  .  .  .  .  .  .  .  . |
 7 |  .  .  .  .  .  ♟  .  . |
 6 |  .  .  .  .  .  .  .  . |
 5 |  .  .  .  .  ♚  .  .  . |
 4 |  .  .  .  .  .  .  .  . |
 3 |  .  .  .  .  .  .  ♔  . |
 2 |  .  .  .  .  .  .  .  . |
 1 |  .  .  .  .  .  .  .  . |
    -------------------------
      a  b  c  d  e  f  g  h 


False

In [ ]:
total_steps += 1
            # white moves
            moves = env.possible_moves
            m = random.choice(moves)
            a = env.move_to_action(m)
            # perform action
            state, reward, done, _ = env.step(a)
            total_rewards["WHITE"] += reward
            if done:
                break
    
            # black moves
            moves = env.possible_moves
            m = random.choice(moves)
            a = env.move_to_action(m)
            # perform action
            state, reward, done, _ = env.step(a)
            total_rewards["BLACK"] += reward
            if done:
                break

    print(">" * 5, "GAME", i, "REWARD:", total_rewards)
    collected_rewards.append(total_rewards)